<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Session 2:  Data Cleaning and Formatting in Python
<a id='beginning'></a>

Having collected the data does not always allow you to produce some analytics right away. There is often a lot of pre processing to be done. 

This session is about:

* Cleaning: making sure each cell has a value that could be used in your coming procedures. The _impurities_ do not allow formatting the data correctly: commas instead of points and viceversa, blanks/spaces, unneeded symbols (dollar, euro symbols), or non-standard symbols to represent missing values.

* Fomatting: making sure the clean value is in the right data type. if you are going to do text analysis, you may need to get rid of repetitive words, normalize them into lower case, and turn them back to their root or stem. For statistical work, you need to differentiate among nominal, ordinal, numerical and strings.

Take a look at the tables in this wikipedia page, pay attention to:

- The values of each index.
- The table of indices for all countries measured.

There is a table on disputed territories, you can skip that one.

In [ ]:
from IPython.display import IFrame  
wikiLink="https://en.wikipedia.org/wiki/List_of_freedom_indices" 
IFrame(wikiLink, width=900, height=500)

You should have observed the following:

1. The values of each index:
   - Varies from three to five **level** values. 
   - They use different **level** names.
   - The levels are **ordinal**.
2. The table of countries:
   - The column names has very long **column names**. 
   - The column names include **footnotes**.
   - The presence of **images** (the flags) in the country column.
   - The use on "n/a" for **missing values**.

Let's try to get the table using pandas:

In [ ]:
import pandas as pd

wikiTables=pd.read_html(wikiLink, # link
                        #header=0, # where is the header?
                        flavor='bs4', # helper to translate html
                        attrs={'class': 'wikitable sortable'}) # attributes to identify element(s)

The object **wikiTables** is a list. I know before hand that it has only one table as data frame:

In [ ]:
wikiTables[0] # you should vary this index until you get the table you need!

Pandas will show yo the _head_ and the _tail_ of the dataframe. There you can verify that the column names are well positioned. Let me save the data frame temporarily:

In [ ]:
DF=wikiTables[0]

From the previous preview, you know that: 
1. Cleaning: You need to rename the columns. You will learn pandas' **rename()**.
2. Cleaning: You need to verify if the column _Country_ is clean: What was caused by the presence of flags?
3. Cleaning: Verify that level names are consistent: the _same_ level may have a slightly different label. You will learn  **value_counts()**.
4. Formatting: Check what data types have been recognised. Use **describe()** and/or **info**().
4. Formatting: Put each column in the right data type if needed.

Notice that Pandas was able to detect the poorly labelled missing value 'n\a' and turned it into the right value: **NaN**.

Let's start!

As you will be changing the data frame, make a copy first: 

In [ ]:
DFTest=DF.copy() # working with a copy...

Continue....

##  **Renaming columns**

Headers names should be simple, no *blanks*, no special characters, no *footnote* calls. We have that problem. You can try these strategies:
1. Using __Brute-force__ strategy: I simply change the names. Very practical when you have few names.

In [ ]:
# all columns
DFTest.columns=['Country', 'FreedomintheWorld', 'IndexofEconomicFreedom','PressFreedomIndex', 'DemocracyIndex']
#see changes:
DFTest.head()

2.  Using **rename()** from Pandas: The advantage is that you can change some, not all, names.

In [ ]:
DFTest=DF.copy() # recreating DFTest
some=['FreedomintheWorld', 'IndexofEconomicFreedom','PressFreedomIndex', 'DemocracyIndex']


In [ ]:
DFTest.columns[1::] # all but the first.

Here, we create a dictionary where the **key** is the original name, and the **value** is the new value:

In [ ]:
dict(zip(DFTest.columns[1::], some))

Now, you are ready to rename:

In [ ]:
DFTest.rename(columns=dict(zip(DFTest.columns[1::], some)))

The previous result is just a preview, no change has been made:

In [ ]:
DFTest

The _rename()_ function, as other pandas functions, has the **inplace** attribute. 

In [ ]:
DFTest.rename(columns=dict(zip(DFTest.columns[1::], some)),inplace=True)

Take a look now:

In [ ]:
DFTest

3. Enter __Computational thinking in cleaning__

When dealing with bigger data sets, you need to get used to automating the cleaning process. It is not possible 100%, but the more you can automate, the better. In general, you will have some strategies for cleaning, which can be used alone or in tandem:

3.1. Regular expression or _REGEX_ approach.

3.2. Partitioning or _Divide and Conquer_ approach.

There are also some particular functions from base Python you will learn on the way.

**<font color="blue">3.1 REGEX APPROACH</font>** 

This approach will use some special coding to extract or delete particular pieces of text. You will first need to call this library, before anything else:

In [ ]:
import re

Let's show you how this works using this example string:

In [ ]:
textExample="var1 iab22 les333[4]"

In the text above, I just need to keep: "**variables**":

In [ ]:
# one or more blanks: \\s+
# one or more numbers: \\d+
# find opening bracket : \\[
# find closing bracket: \\]

# You can combine using '|' (or):
WhenYouFind='\\s+|\\d+|\\[|\\]'
replaceWith=''

# use it:
re.sub(WhenYouFind,replaceWith,textExample)

I use the function in one string, now I want to use it in a group of elements, the column names, **would it work**??

In [ ]:
# this will not work:
#re.sub(WhenYouFind,replaceWith,DFTest.columns)

This is a list comprehension:

In [ ]:
# substitute the elements in each NAME in the COLUMNS:
[re.sub(WhenYouFind,replaceWith,NAME) for NAME in DFTest.columns]

I can concatenate other functions:

In [ ]:
# Turn the result to lower case
[re.sub(WhenYouFind,replaceWith,NAME).lower() for NAME in DFTest.columns]

If you know you will often use this transformation, you can create a __function__: 

In [ ]:
# "lambda" style function
# name of function is 'cleaner'
# function transforms INPUT

cleaner = lambda INPUT: re.sub(WhenYouFind,replaceWith,INPUT).lower()

You can now use this function in the comprehension:

In [ ]:
[cleaner(TEXT) for TEXT in DF.columns]

You can use Pandas own **str** functions:

In [ ]:
DFTest=DF.copy() # recreating DFTest

# Pandas only:
DFTest.columns.str.replace(WhenYouFind,replaceWith,regex=True).str.lower()

Then, I just clean those ones:

In [ ]:
DFTest.columns[1::].str.replace(WhenYouFind,replaceWith,regex=True).str.lower()

In [ ]:
# saving result :
someNewNames=DFTest.columns[1::].str.replace(WhenYouFind,replaceWith,regex=True).str.lower()
# which are:
someNewNames

Now that you have clean last names, it is time to change the old ones. First, make sure you have a clean one for each bad one:

In [ ]:
# using 'zip()':
changes=dict(zip(DF.iloc[:,1::].columns,someNewNames))
changes

Once you have a dict like that one, you can use it to rename the columns as before with *rename()*:

In [ ]:
DFTest.rename(columns=changes,inplace=True)

Let's see the result:

In [ ]:
DFTest

**<font color="blue">3.2.  DIVIDE AND CONQUER</font>** 

This is an strategy that works in tandem with the previous ones. It is used to reduce complexity of the tasks.

In [ ]:
DFTest=DF.copy() # recreating DFTest
DFTest.columns

In [ ]:
# split the text when you find '['
notSoBad=[TEXT.split("[") for TEXT in DFTest.columns]
# see:
notSoBad

In [ ]:
# split the text when you find '['and keep first element:
notSoBad=[TEXT.split("[")[0] for TEXT in DFTest.columns]
# see:
notSoBad

In [ ]:
#split using the blanks
goodEnough=[TEXT.split(" ") for TEXT in notSoBad]
# see:
goodEnough

In [ ]:
#split using the blanks and them concatenate using "join()":
betterThanBad=["".join(TEXT.split(" ")) for TEXT in notSoBad]
# see:
betterThanBad

Let me know get rid of the year, and the words _index_ and *of*:

In [ ]:
#original:
import re
[re.sub('\d+|index|of',"",text.lower()) for text in betterThanBad]

Just changing the names:

In [ ]:
DF.columns=[re.sub('\d+|index|of',"",text.lower()) for text in betterThanBad]

##  **Check country names** 

As you may have realized, the cells have categories, so we do not need to compute statistics, as they will not say much. Let me take a loom at the country cells:

In [ ]:
DF.country[0]

The column with country names is very important for merging (this is the **key** of the unit of analysis). You should always do this:

In [ ]:
# no trailing nor leading spaces:
DF['country']=DF.country.str.strip()

You can apply the same function to every column:

In [ ]:
byeSpaces= lambda COLUMN:COLUMN.str.strip()

DF.apply(byeSpaces)

In [ ]:
# save the result:
DF=DF.apply(byeSpaces)

## Verify that level names are consistent

Let's keep looking for mistakes. Sometimes a category may be wrongly written in a cell. For instance, if you had 'Free' and 'free' or 'free ' to represent the same value in one column, you have a mistake. Let's see if there is one here. First, learn how to use **value_counts**:

In [ ]:
DF.freedomintheworld.value_counts()

Let's do the same for all the columns at once:

In [ ]:
[DF[COLUMN].value_counts().index for COLUMN in DF.iloc[:,1::]]

The values of each cell are clean. So no more work to be done here.

## Check the data types

First, see what Python has assigned:

In [ ]:
DF.dtypes

Every column is type **object**. That is OK for country, but not for the others. Let's make the changes:

a. Recognise the levels.

b. Create the data type.

c. If data is ordinal, set it so.

We already know the levels (step a) when we use **value_counts()**. Let me make the first change:

In [ ]:
from pandas.api.types import CategoricalDtype

# create data type
levelsFree = CategoricalDtype(categories=["not free", "partly free", "free"], ordered=True)
# make the change:
DF.freedomintheworld=DF.freedomintheworld.astype(levelsFree)

See the changes:

In [ ]:
DF.freedomintheworld

You can reorder the position of the labels to avoid writing text:

In [ ]:
#original
DF.economicfreedom.value_counts().index.tolist()

In [ ]:
oldEcon = DF.economicfreedom.value_counts().index.tolist()
newPositions = [3, 1,0,2,4]
newEcon = [oldEcon[i] for i in newPositions]
newEcon

Get the other's new order:

In [ ]:
oldPress = DF.pressfreedom.value_counts().index.tolist()
newPositions = [3, 1,0,2,4]
newPress = [oldPress[i] for i in newPositions]

oldDemo = DF.democracy.value_counts().index.tolist()
newPositions = [1, 2,0,3]
newDemo = [oldDemo[i] for i in newPositions]

Create the data types:

In [ ]:
levelsEcon = CategoricalDtype(categories=newEcon, ordered=True)
levelsPress = CategoricalDtype(categories=newPress, ordered=True)
levelsDemo = CategoricalDtype(categories=newDemo, ordered=True)

Make all changes:

In [ ]:
DF.economicfreedom=DF.economicfreedom.astype(levelsEcon)
DF.pressfreedom=DF.pressfreedom.astype(levelsPress)
DF.democracy=DF.democracy.astype(levelsDemo)

The result:

In [ ]:
DF.dtypes

In [ ]:
DF.describe()

Notice that you have categorical data with text as levels:

In [ ]:
DF

It would be a good idea to have numbers instead. You can use **cat.codes** from Pandas:

In [ ]:
getCodes = lambda COLUMN:COLUMN.cat.codes

In [ ]:
# a new one
DFnum=DF.copy() 

# get the codes as numeric
DFnum.iloc[:,1::]=DF.iloc[:,1::].apply(getCodes)

# see result:
DFnum

Notice that the missing values are represented by **-1**. We can set the missing values with the help of **numpy**:

In [ ]:
import numpy as np

DFnum.replace({-1:np.nan},inplace=True)

#result:
DFnum

Remember that _economicfreedom_ and _pressfreedom_ have five values, while _freedomintheworld_ has three, and _democracy_ has four. Let's make those ranges compatibles:

In [ ]:
replacing1={1:2,2:4}
DFnum.freedomintheworld.replace(replacing1,inplace=True)
replacing2={2:3,3:4}
DFnum.democracy.replace(replacing2,inplace=True)

In [ ]:
DFnum

Let's check general information:

In [ ]:
DFnum.describe()

In [ ]:
DFnum.info()

Even though you have numbers, these can be categories:

In [ ]:
# create data type
levelsALL = CategoricalDtype(categories=[0,1,2,3,4], ordered=True)

In [ ]:
#make changes:
DFnum.iloc[:,1::]=DFnum.iloc[:,1::].astype(levelsALL)

Becareful with the last operation, because you create unused levels:

In [ ]:
DFnum.freedomintheworld.value_counts()

You can solve that here:

In [ ]:
byeCats= lambda COLUMN:COLUMN.cat.remove_unused_categories()
DFnum.iloc[:,1::]=DFnum.iloc[:,1::].apply(byeCats)

Verify the changes:

In [ ]:
DFnum.freedomintheworld.value_counts()

In [ ]:
DFnum.democracy.value_counts()

## Exercise

Take a look at this [file](https://github.com/UWDataScience2020/data/raw/master/HDI_2018.xlsx). WRITE DOWN the algorithm you will follow to have it clean.